# Examples with JSON Log Entries

> **These are Go notebooks**: In order to access the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## Saving logs (standard)

When running larger or more complex simulations it sometimes makes sense to simply log the generated data in some file for later analysis.

WIP

In [ ]:
import "github.com/umbralcalc/stochadex/pkg/simulator"

%%

// How do I manually setup a simulation config?
generator := simulator.NewConfigGenerator()

// How do I manually configure a simulation state partition?
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "first_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0, 3.0, 4.0},
	}),
	InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              12345,
})

// How do I manually configure another simulation state partition?
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "second_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0},
	}),
	InitStateValues:   []float64{0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              5678,
})

// How do I manually configure the extra simulation run specs?
generator.SetSimulation(&simulator.SimulationConfig{
	OutputCondition: &simulator.EveryStepOutputCondition{},
	OutputFunction: simulator.NewJsonLogOutputFunction("./data/test.log"),
	TerminationCondition: &simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	TimestepFunction: &simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	InitTimeValue: 0.0,
})

// How should setup a simulation with this configuration?
coordinator := simulator.NewPartitionCoordinator(generator.GenerateConfigs())

// How do I run the simulation?
coordinator.Run()

## Saving logs (advanced)

WIP

In [ ]:
import "github.com/umbralcalc/stochadex/pkg/simulator"

%%

// How do I manually setup a simulation config?
generator := simulator.NewConfigGenerator()

// How do I manually configure a simulation state partition?
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "first_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0, 3.0, 4.0},
	}),
	InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              12345,
})

// How do I manually configure another simulation state partition?
generator.SetPartition(&simulator.PartitionConfig{
	Name:              "second_wiener_process",
	Iteration:         &continuous.WienerProcessIteration{},
	Params:            simulator.NewParams(map[string][]float64{
		"variances": {1.0, 2.0},
	}),
	InitStateValues:   []float64{0.0, 0.0},
	StateHistoryDepth: 1,
	Seed:              5678,
})

// How do I create a higher performance logging channel?
logChannel := simulator.NewJsonLogChannelOutputFunction("./data/test.log")
defer logChannel.Close()

// How do I manually configure the extra simulation run specs?
generator.SetSimulation(&simulator.SimulationConfig{
	OutputCondition: &simulator.EveryStepOutputCondition{},
	OutputFunction: logChannel,
	TerminationCondition: &simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	TimestepFunction: &simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	InitTimeValue: 0.0,
})

// How should setup a simulation with this configuration?
coordinator := simulator.NewPartitionCoordinator(generator.GenerateConfigs())

// How do I run the simulation?
coordinator.Run()

## Loading logs

In [ ]:
import (
	"strconv"

	"github.com/umbralcalc/stochadex/pkg/analysis"

	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%
// a log of this kind can be generated by running ./bin/stochadex --config ./cfg/config.yaml
storage, _ := analysis.NewStateTimeStorageFromJsonLogEntries("./data/test.log")
yRefs := make([]analysis.PlotDataRef, 0)
for i := 0; i < 4; i++ {
	yRefs = append(yRefs, analysis.PlotDataRef{
		PartitionName: "first_wiener_process",
		ValueIndex: i,
	})
}

scatter := analysis.NewScatterPlotFromPartition(
	storage,
	analysis.PlotDataRef{
		PartitionName: "first_wiener_process",
		IsTime: true,
	},
	yRefs,
)

gonb_echarts.Display(
	scatter, 
	"width: 1024px; height:400px; background: white;",
)